In [8]:
#Do some setup
from google import genai
import google.generativeai as genais
from google.genai.types import HttpOptions

from IPython.display import HTML, Markdown, display
from google.api_core import retry
import os

# Import environment variables from env.json
import json
with open('../../env.json', 'r') as f: #TODO: fix
    env_vars = json.load(f)
# Set environment variables from the loaded file
os.environ["GOOGLE_CLOUD_PROJECT"] = env_vars["google_cloud_project"]
os.environ["GOOGLE_CLOUD_LOCATION"] = env_vars["google_cloud_location"]
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = env_vars["google_genai_use_vertexai"]
# Set the fine-tuned model ID as an environment variable
os.environ["FINE_TUNED_MODEL_V1_ID"] = env_vars["fine_tuned_v1_model_id"]
os.environ["FINE_TUNED_MODEL_V2_ID"] = env_vars["fine_tuned_v2_model_id"]
os.environ["GOOGLE_API_KEY"] = env_vars["google_api_keys"][1]
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

genais.configure(api_key=GOOGLE_API_KEY)


In [9]:
import pandas as pd
# Define the path to the CSV file
csv_path = "../../data/Prompt_Training_2.0/seperated_test_data.csv"

# Load the CSV file into a pandas DataFrame
test_df = pd.read_csv(csv_path)
display(test_df)

,original_prompt,context,instruction,has_context,conversation_id
0,One-pot vegetarian pasta recipes for busy nights,NaN,One-pot vegetarian pasta recipes for busy nights,False,93453
1,We have the following blog content... what is ...,"The content:\n\nConsumers want more choices, b...",We have the following blog content... what is ...,True,65263
2,how o sort element using merge sort technique ...,NaN,how o sort element using merge sort technique ...,False,99000
3,"make a javascript class ""GraphicLayer"" which i...","One of those properties will be center point, ...","make a javascript class ""GraphicLayer"" which i...",True,96296
4,Please outline the steps to build an automated...,NaN,Please outline the steps to build an automated...,False,38806
...,...,...,...,...,...
957,Write a thread of tweets with some advice abou...,NaN,Write a thread of tweets with some advice abou...,False,69371
958,Create message to real estate agent saying you...,NaN,Create message to real estate agent saying you...,False,85308
959,"How much hardware, energy, etc. are required f...",NaN,"How much hardware, energy, etc. are required f...",False,98052
960,"As CFA-Sensei, a CFA Level 1 bot, your role is...",1.\tEthical and Professional Standards\n2.\tQu...,"As CFA-Sensei, a CFA Level 1 bot, your role is...",True,26124


In [22]:
# import models
base_model = genais.GenerativeModel('gemini-2.0-flash-lite-001')  # base untuned model

# Access the model ID from environment variables
fine_tuned_model_v1 = os.environ["FINE_TUNED_MODEL_V1_ID"]
fine_tuned_model_v2 = os.environ["FINE_TUNED_MODEL_V2_ID"]
# Initialize the GenAI client for Vertex AI
client = genai.Client(http_options=HttpOptions(api_version="v1"))
api = client.models
# Define constants for model types
Base, V1, V2 = 0, 1, 2

def generate_prompt(model: int, outline):
    try: 
        """
        Generate content using the specified model.
        
        Args:
            outline: The prompt text to generate content from
            model: Integer indicating which model to use (BASE=0, V1=1, V2=2)
        
        Returns:
            Generated text content
        """
        if model == Base:
            return base_model.generate_content(outline).text
        elif model == V1:
            return api.generate_content(
                model=fine_tuned_model_v1,
                contents=outline,
            ).text
        elif model == V2:
            return api.generate_content(
                model=fine_tuned_model_v2,
                contents=outline
            ).text
        else:
            raise ValueError(f"Invalid model type: {model}. Use Base (0), V1 (1), or V2 (2).")
    except Exception as e:
        print(f"Error processing prompt: {str(e)[:100]}...")
        return "Error generating response"
    
def get_outline(model: int, prompt='', instruction='', context=''):
    if model == Base or model == V1:
        return f'''You are a prompt engineering expert that transforms simple 
        prompts into more effective versions. Analyze the input prompt and create
        an improved version that includes specific details, context, desired output format, 
        and any relevant constraints. Make the prompt clear, specific, and designed to 
        generate high-quality responses.
        
        Input Prompt: {prompt}

        Respond ONLY with the text of the improved prompt, without any explanations, 
        introductions, or additional commentary.
        '''
    elif model == V2:
        return f"""You are a prompt engineering expert.
        Your task is to rewrite the instruction below using advanced prompt engineering techniques. If context is provided, use it as *background knowledge* to better understand the task — but do not include it in the final output.

        Guidelines:
        - Enhance the instruction to be clearer, more specific, and more effective
        - Use any prompting technique that best fits
        - Ground your rewrite in the provided context, if applicable
        - Do NOT copy or reference the context in your rewritten instruction

        Context:
        {context}

        Original Instruction:
        {instruction}

        Output ONLY the improved instruction without any additional text, titling, explanations, or acknowledgment."""
    else: 
        raise ValueError(f"Invalid model type: {model}. Use Base (0), V1 (1), or V2 (2).")

In [30]:
from tqdm import tqdm

path = '../../data/Responses'
os.makedirs(path, exist_ok=True)

# Create a copy of the dataframe to store results
results_df = test_df[:10].copy()

# Iterate through the dataframe rows with index
for idx, row in tqdm(results_df.iterrows()):
    # Access row values using the row Series
    results_df.at[idx, 'Base_Prompt'] = generate_prompt(Base, get_outline(Base, row['original_prompt']))
    results_df.at[idx, 'V1_Prompt'] = generate_prompt(V1, get_outline(V1, row['original_prompt']))
    results_df.at[idx, 'V2_instruction'] = generate_prompt(V2, get_outline(V2, context=row['context'], instruction=row['instruction']))

# Save the results
results_df.to_csv(path + '/Prompt_Examples.csv', index=False)   
display(results_df)


8it [01:07,  7.52s/it]

Error processing prompt: [Errno 11001] getaddrinfo failed...
Error processing prompt: [Errno 11001] getaddrinfo failed...


10it [01:48, 10.82s/it]


,original_prompt,context,instruction,has_context,conversation_id,Base_Prompt,V1_Prompt,V2_instruction
0,One-pot vegetarian pasta recipes for busy nights,NaN,One-pot vegetarian pasta recipes for busy nights,False,93453,Create a list of 5 easy one-pot vegetarian pas...,I want to create a one-pot vegetarian pasta co...,"Generate 10 easy-to-follow, one-pot vegetarian..."
1,We have the following blog content... what is ...,"The content:\n\nConsumers want more choices, b...",We have the following blog content... what is ...,True,65263,Analyze the provided blog content from GigSala...,Here's an example of how to analyze blog conte...,You are an expert at inferring and understandi...
2,how o sort element using merge sort technique ...,NaN,how o sort element using merge sort technique ...,False,99000,Create a Java program that implements the merg...,Imagine you're a senior developer tasked with ...,Assume the role of an experienced Java develop...
3,"make a javascript class ""GraphicLayer"" which i...","One of those properties will be center point, ...","make a javascript class ""GraphicLayer"" which i...",True,96296,"```\nCreate a JavaScript class named ""GraphicL...",```javascript\nclass GraphicLayer {\n constru...,Generate a JavaScript class named `GraphicLaye...
4,Please outline the steps to build an automated...,NaN,Please outline the steps to build an automated...,False,38806,"Develop a comprehensive, step-by-step plan to ...",You are an expert chatbot architect. Your task...,Assume the persona of a highly technical AI So...
5,Write a tweet announcing the launch of SWR 2.0...,NaN,Write a tweet announcing the launch of SWR 2.0...,False,61132,Write a tweet announcing the launch of SWR 2.0...,"I am going to provide you with example code, a...","Assume the role of a vibrant, technical commun..."
6,One-pot vegetarian pasta recipes for busy nights,NaN,One-pot vegetarian pasta recipes for busy nights,False,72230,"Develop five concise, step-by-step recipes for...",You are a professional chef specializing in ve...,"Generate a list of 5-7 simple, 30-minute one-p..."
7,We have the following blog content... what is ...,"The content:\n\nConsumers want more choices, b...",We have the following blog content... what is ...,True,53181,Analyze the provided blog content and determin...,Error generating response,Error generating response
8,how o sort element using merge sort technique ...,NaN,how o sort element using merge sort technique ...,False,12179,Implement a Java program to sort an array of i...,You are an expert computer science instructor....,Write a Java program demonstrating the Merge S...
9,"make a javascript class ""GraphicLayer"" which i...","One of those properties will be center point, ...","make a javascript class ""GraphicLayer"" which i...",True,21009,"```\nCreate a JavaScript class named ""GraphicL...",You are an expert software engineer tasked wit...,Develop a JavaScript class named `GraphicLayer...


In [33]:
import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')

def get_response(prompt):
    response = model.generate_content(prompt)
    return response.text

for idx, row in tqdm(results_df.iterrows()):
    # Access row values using the row Series
    results_df.at[idx, 'No_Model_Response'] = get_response(str(row['original_prompt']))
    results_df.at[idx, 'Base_Response'] = get_response(str(row['Base_Prompt']))
    results_df.at[idx, 'V1_Response'] = get_response(str(row['V1_Prompt']))
    results_df.at[idx, 'V2_Response'] = get_response(str(row['V2_instruction']) + '\n' + str(row['context']))

# Save the results
results_df.to_csv(path + '/Response_Examples.csv', index=False)   
display(results_df)


10it [07:16, 43.69s/it]


,original_prompt,context,instruction,has_context,conversation_id,Base_Prompt,V1_Prompt,V2_instruction,No_Model_Response,Base_Response,V1_Response,V2_Response
0,One-pot vegetarian pasta recipes for busy nights,NaN,One-pot vegetarian pasta recipes for busy nights,False,93453,Create a list of 5 easy one-pot vegetarian pas...,I want to create a one-pot vegetarian pasta co...,"Generate 10 easy-to-follow, one-pot vegetarian...","Okay, here are some one-pot vegetarian pasta r...",Here are 5 easy one-pot vegetarian pasta recip...,Here are 5 one-pot vegetarian pasta recipe pro...,Here are 10 easy one-pot vegetarian pasta reci...
1,We have the following blog content... what is ...,"The content:\n\nConsumers want more choices, b...",We have the following blog content... what is ...,True,65263,Analyze the provided blog content from GigSala...,Here's an example of how to analyze blog conte...,You are an expert at inferring and understandi...,"Okay, let's break down the likely user intent ...","**Prompt:**\n\n""You are an expert in event pla...","Okay, I understand. I'll provide the blog cont...",1. **Understand the trade-offs between cost a...
2,how o sort element using merge sort technique ...,NaN,how o sort element using merge sort technique ...,False,99000,Create a Java program that implements the merg...,Imagine you're a senior developer tasked with ...,Assume the role of an experienced Java develop...,```java\npublic class MergeSort {\n\n publi...,```java\npublic class MergeSort {\n\n publi...,"Okay, let's break down merge sort, write the J...",```java\npublic class MergeSort {\n\n /**\n...
3,"make a javascript class ""GraphicLayer"" which i...","One of those properties will be center point, ...","make a javascript class ""GraphicLayer"" which i...",True,96296,"```\nCreate a JavaScript class named ""GraphicL...",```javascript\nclass GraphicLayer {\n constru...,Generate a JavaScript class named `GraphicLaye...,```javascript\nclass GraphicLayer {\n const...,```javascript\nclass GraphicLayer {\n /**\n...,This is a well-structured and functional JavaS...,```javascript\nclass GraphicLayer {\n const...
4,Please outline the steps to build an automated...,NaN,Please outline the steps to build an automated...,False,38806,"Develop a comprehensive, step-by-step plan to ...",You are an expert chatbot architect. Your task...,Assume the persona of a highly technical AI So...,"Okay, here's a breakdown of the steps to build...",Here's a comprehensive plan for building an au...,"```json\n{\n ""steps"": [\n {\n ""title""...","Alright, buckle up. We're about to dive deep i..."
5,Write a tweet announcing the launch of SWR 2.0...,NaN,Write a tweet announcing the launch of SWR 2.0...,False,61132,Write a tweet announcing the launch of SWR 2.0...,"I am going to provide you with example code, a...","Assume the role of a vibrant, technical commun...","SWR 2.0 IS HERE! 🚀 Faster, leaner, and even mo...",SWR 2.0 is HERE! 🚀 Blazing fast data fetching ...,```javascript\nfunction SWRv2Features() {\n c...,SWR 2.0 is finally here! 🎉 #SWR2\n\n◆ Faster t...
6,One-pot vegetarian pasta recipes for busy nights,NaN,One-pot vegetarian pasta recipes for busy nights,False,72230,"Develop five concise, step-by-step recipes for...",You are a professional chef specializing in ve...,"Generate a list of 5-7 simple, 30-minute one-p...","Okay, here are some delicious and easy one-pot...","Okay, here are five concise, one-pot vegetaria...",## One-Pot Creamy Tomato & Spinach Pasta (Vege...,"Okay, here are 6 simple, 30-minute one-pot pas..."
7,We have the following blog content... what is ...,"The content:\n\nConsumers want more choices, b...",We have the following blog content... what is ...,True,53181,Analyze the provided blog content and determin...,Error generating response,Error generating response,"Okay, based on the content, here's a breakdown...","Okay, let's analyze the user intent and craft ...",I am sorry I was unable to generate a response...,## 6 Reasons to Choose the Best E